# Import Auxiliary Libraries

In [ ]:
import numpy as np
import pandas as pd

# Load Dataset

In [ ]:
# Load data
from sklearn import datasets
bc_data = datasets.load_breast_cancer()                                             # Loading from Scikit Learn


dataset = pd.DataFrame(bc_data.data, index = np.arange(len(bc_data.data)))          # Bunch object to pandas dataframe also, assigning index
dataset['target'] = bc_data.target                                                  # Appending target attribute to the dataframe from array

dataset.head()                                                                      # Glimpse of the dataset - top 5 rows

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


# Reshaping, Reconfiguring the dataset and Splitting it in train and test set having 20% test data

In [ ]:
X = dataset.iloc[:,:-1].values                                                      # Selecting the features of the data and turing it into numpy.ndarray for computational purpose
y = dataset.iloc[:,-1].values.reshape(-1,1)                                         # Selecting the target attribute and asking numpy to match the row number with having a single column               

np.random.seed(123)                                                     
test_fraction = 0.20                                                                # Defining the test size of the data
test_size = int(len(y)*test_fraction)
test_idxs = np.random.choice(np.arange(len(y)), test_size, replace = False)         # Getting random indices to slice test set
i = int((1 - test_fraction) * X.shape[0]) 
o = np.random.permutation(X.shape[0])   
X_train, X_test = np.split(np.take(X,o,axis=0), [i])                                # Assigning train and test for features
y_train, y_test = np.split(np.take(y,o), [i])                                       # Assigning train and test for target

# Helper Functions

In [ ]:
# Loss Funtion
def entropy(y):
    size = len(y)
    classes, counts = np.unique(y, return_counts = True)
    probability = counts/size
    return -np.sum(probability*np.log2(probability))

# Node Class

In [ ]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):                # Constructor
        # For Buds
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

        # For terminal node
    def is_leaf_node(self):
        return self.value is not None

In [ ]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=2, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    def fit(self, X, y):                                                                                # Method to apply the algorithm to the dataset
        self.n_feats = X.shape[1]                                                                       # Assigning the number of features
        self.root = self._grow_tree(X, y)

    def predict(self, X):                                                                               # Method to predict accroding to the training
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _grow_tree(self, X, y, depth=0):                                                                # Method to build the tree
        n_samples, n_features = X.shape                                                                 # Defining the number of samples and features
        n_labels = len(np.unique(y))                                                                    # Number of labels in the target attribute

        # stopping criteria
        if (
            depth >= self.max_depth
            or n_labels == 1
            or n_samples < self.min_samples_split
        ):
            leaf_value = self._most_common_label(y)                                                     # If condition is fulfilled, it is a terminal/leaf node
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedily select the best split according to information gain
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)                                   # Best feature and best thrshold to split the tree

        # grow the children that result from the split
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -float("inf")                                                                        # best gain set to minus infinity so that any value can be greater than this
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:                                                                       # Iterating over features to find the best split
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:                                                                 # Checking information gain for all possible thresholds
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:                                                                     # Comparing information gain with every iteration
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):                                              # Calculating information gain
        # parent loss
        parent_entropy = entropy(y)                                                                      # Parent entropy

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r                                               # Child entropy

        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy                                                             # information gain
        return ig

    def _split(self, X_column, split_thresh):                                                           # Funtion to split dataset based on condition
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):                                                                  # Tree traversing function
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):                                                                    # Calculating the maximum class for terminal node
        y = list(y)
        return max(y, key=y.count)

# Performance Measurement Helper Function

In [ ]:
def accuracy(true, pred):
        accuracy = np.sum(true == pred) / len(true)
        return accuracy*100

# Running the model on the dataset

In [ ]:
clf = DecisionTree(max_depth=3)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
x_pred = clf.predict(X_train)

# Training Pefromance

In [ ]:
train_acc = accuracy(y_train, x_pred)
print("Accuracy:", train_acc)

Accuracy: 96.7032967032967


# Testing Perfromance

In [ ]:
test_acc = accuracy(y_test, y_pred)
print("Accuracy:", test_acc)

Accuracy: 91.22807017543859
